In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from itertools import *

import pandas as pd
import numpy as np
import random

import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
name = "kcy"
num = 24
DATA_PATH = name + "_df" + str(num) + ".csv"
df = pd.read_csv(DATA_PATH)

In [3]:
params = {"criterion" : ["gini", "entropy", "log_loss"],
          "splitter" : ["best", "random"],
          "max_features" : [1, "auto", "sqrt", "log2"]}
epochs = 50

In [4]:
strt = time.time()
final_score_lst = []
for _ in range(epochs):
    print(str(_ + 1) + " Epoch starts working")
    df = df.sample(frac = 1)
    point1 = int(len(df) * 0.5)
    train, test = df.iloc[:point1, :], df.iloc[point1: , :]
    score_dict = dict()
    for _idx in range(100):
        
        point2 = int(len(train) * 0.5)
        trainA, trainB = train.iloc[:point2, :], train.iloc[point2:, :]
        trainA = trainA.sample(frac = 1); trainB = trainB.sample(frac = 1)
        
        X_train, y_train = trainA.drop(["target"], axis = 1), trainA["target"]
        X_test, y_test = trainB.drop(["target"], axis = 1), trainB["target"]
        
        dt_clf = DecisionTreeClassifier()
        model = GridSearchCV(dt_clf, params, cv = 5, refit = True)
        model.fit(X_train, y_train)
        
        fmodel = model.best_estimator_

        for col in ["None"] + list(X_train.columns):
            if col == "None":
                pred = fmodel.predict(X_test)
                if _idx == 0: score_dict["og"] = accuracy_score(y_test, pred)
                else: score_dict["og"] += accuracy_score(y_test, pred)
            else:
                save = X_test[col].copy()
                X_test[col] = np.random.permutation(X_test[col])
                pred = fmodel.predict(X_test)
                X_test[col] = save
                if _idx == 0: score_dict[col] = accuracy_score(y_test, pred)
                else: score_dict[col] += accuracy_score(y_test, pred)
    lst = []
    for k, v in score_dict.items():
        if k == "og": key = v
        else:
            lst.append([k, (key - v) / 100])
            
    lst = sorted(lst, key = lambda x : x[1], reverse = True)
    for a, b in enumerate(lst):
        if b[1] < 0: break
        else: lst[a] = lst[a][0]
    if a == 0: pass
    else: lst = lst[:a]
    
    train = train.loc[:, lst + ["target"]]; test = test.loc[:, lst + ["target"]]
    
    X, y = train.drop(["target"], axis = 1), train["target"]
    X_test, y_test = test.drop(["target"], axis = 1), test["target"]
    
    dt_clf = DecisionTreeClassifier()
    model = GridSearchCV(dt_clf, params, cv = 5, refit = True)
    model.fit(X, y)
    fmodel = model.best_estimator_
    
    pred = fmodel.predict(X_test)
    
    final_score_lst.append([len(lst), accuracy_score(pred, y_test)])
end = time.time()

1 Epoch starts working
2 Epoch starts working
3 Epoch starts working
4 Epoch starts working
5 Epoch starts working
6 Epoch starts working
7 Epoch starts working
8 Epoch starts working
9 Epoch starts working
10 Epoch starts working
11 Epoch starts working
12 Epoch starts working
13 Epoch starts working
14 Epoch starts working
15 Epoch starts working
16 Epoch starts working
17 Epoch starts working
18 Epoch starts working
19 Epoch starts working
20 Epoch starts working
21 Epoch starts working
22 Epoch starts working
23 Epoch starts working
24 Epoch starts working
25 Epoch starts working
26 Epoch starts working
27 Epoch starts working
28 Epoch starts working
29 Epoch starts working
30 Epoch starts working
31 Epoch starts working
32 Epoch starts working
33 Epoch starts working
34 Epoch starts working
35 Epoch starts working
36 Epoch starts working
37 Epoch starts working
38 Epoch starts working
39 Epoch starts working
40 Epoch starts working
41 Epoch starts working
42 Epoch starts working
4

In [5]:
v, s = 0, 0
for a, b in final_score_lst:
    v += a; s += b
print(v / epochs, s / epochs, end - strt)

10.74 0.6436079999999998 2172.928414106369


In [6]:
temp_lst = [i[1] for i in final_score_lst]
print(np.mean(temp_lst), np.median(temp_lst), np.std(temp_lst))

0.643608 0.6449 0.006989244308221018


In [7]:
pd.DataFrame(final_score_lst).to_csv("PI-res-" + DATA_PATH, index = False)